In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import models
from numpy import array
from tensorflow.keras import layers

In [2]:
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words = vocab_size)
class_names = ["negative", "positive"]

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [3]:
index = tf.keras.datasets.imdb.get_word_index()
index = {k:(v+3) for k,v in index.items()}

index["<PAD>"]=0
index["<START>"]=0
index["<UNKNOWN>"]=0
index["<UNUSED>"]=0


reverse_index = dict([(value, key) for (key, value) in index.items()])
def decode_review(text):
  return ' '.join([reverse_index.get(i, "?") for i in text])


1646592/1641221 [==============================] - 0s 0us/step


In [5]:
review_length = 500
model = models.Sequential()
model.add(
    layers.Embedding(
        input_dim = vocab_size, #10000
        output_dim = 32,
        input_length = review_length #500
    )
)



model.add(layers.Dropout(rate=0.25))
model.add(layers.LSTM(units = 32))
model.add(layers.Dropout(rate=0.25))
model.add(layers.Dense(units=1, activation='sigmoid'))

model.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['Accuracy']
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           320000    
_________________________________________________________________
dropout (Dropout)            (None, 500, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________


In [6]:
review_length = 500

x_train = sequence.pad_sequences(x_train, maxlen = review_length)
x_test = sequence.pad_sequences(x_test, maxlen = review_length)


history = model.fit(
    x_train, y_train, batch_size = 256,
    epochs = 100,
    validation_split = 0.2,
    verbose =1
)

Epoch 1/100
79/79 [==============================] - 46s 547ms/step - loss: 0.6039 - accuracy: 0.0000e+00 - val_loss: 0.4321 - val_accuracy: 0.0000e+00
Epoch 2/100
79/79 [==============================] - 43s 542ms/step - loss: 0.3477 - accuracy: 0.0000e+00 - val_loss: 0.3087 - val_accuracy: 0.0000e+00
Epoch 3/100
79/79 [==============================] - 43s 544ms/step - loss: 0.2455 - accuracy: 0.0000e+00 - val_loss: 0.2933 - val_accuracy: 0.0000e+00
Epoch 4/100
79/79 [==============================] - 43s 541ms/step - loss: 0.1937 - accuracy: 0.0000e+00 - val_loss: 0.3035 - val_accuracy: 0.0000e+00
Epoch 5/100
79/79 [==============================] - 43s 543ms/step - loss: 0.1609 - accuracy: 0.0000e+00 - val_loss: 0.3220 - val_accuracy: 0.0000e+00
Epoch 6/100
79/79 [==============================] - 43s 540ms/step - loss: 0.1362 - accuracy: 0.0000e+00 - val_loss: 0.3207 - val_accuracy: 0.0000e+00
Epoch 7/100
79/79 [==============================] - 43s 540ms/step - loss: 0.1137 - acc

In [7]:
from sklearn.metrics import classification_report
predicted_classes = model.predict_classes(x_test)
print(classification_report(y_test, predicted_classes, target_names=class_names))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

    negative       0.83      0.86      0.85     12500
    positive       0.86      0.83      0.84     12500

    accuracy                           0.84     25000
   macro avg       0.84      0.84      0.84     25000
weighted avg       0.84      0.84      0.84     25000



In [10]:
# Our Review


Review = "This is a very good comedy movie"

review=Review.lower()

tmp = []
for word in review.split(" "):
  tmp.append(index[word])

tmp_padded = sequence.pad_sequences([tmp], maxlen=review_length)

rawprediction = model.predict(array([tmp_padded][0]))[0][0]
prediction = int(round(rawprediction))

print("Review: " + review)
print("Raw Prediction: " + str(rawprediction))
print("Predicted Class: " + class_names[prediction])

Review: this is a very good comedy movie
Raw Prediction: 0.9155137
Predicted Class: positive
